In [1]:
import helpers
import numpy as np

#helpers.npNicePrint(arr)

data = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""

def repeat_and_join(input_string, repeat_count, separation_character):
    repeated_string = input_string * repeat_count
    joined_string = separation_character.join(repeated_string[i:i+len(input_string)] for i in range(0, len(repeated_string), len(input_string)))
    return joined_string


with open("input_12.txt", 'r') as f:
    data = f.read().strip()

#grid = {(x,y): val for y, r in enumerate(data.split('\n')) for x, val in enumerate(r) if val == "#"}
gear_rows = []
for row in data.split("\n"):
    gears, config = row.split(" ")
    config = [int(x) for x in config.split(",")]
    gear_rows.append([repeat_and_join(gears, 5, "?"), config*5])

#start_coord = np.array([k for k,v in grid.items() if v == "S"][0])
gear_rows

[['??????#??#?????????#??#?????????#??#?????????#??#?????????#??#??',
  [1, 1, 5, 1, 1, 1, 5, 1, 1, 1, 5, 1, 1, 1, 5, 1, 1, 1, 5, 1]],
 ['?#?#??##?#???#?#??##?#???#?#??##?#???#?#??##?#???#?#??##?#?',
  [2, 5, 1, 2, 5, 1, 2, 5, 1, 2, 5, 1, 2, 5, 1]],
 ['????????.?#???#??##??????????.?#???#??##??????????.?#???#??##??????????.?#???#??##??????????.?#???#??##?',
  [2,
   1,
   2,
   1,
   1,
   6,
   2,
   1,
   2,
   1,
   1,
   6,
   2,
   1,
   2,
   1,
   1,
   6,
   2,
   1,
   2,
   1,
   1,
   6,
   2,
   1,
   2,
   1,
   1,
   6]],
 ['#???#?????.?#?.?#???#?????.?#?.?#???#?????.?#?.?#???#?????.?#?.?#???#?????.?#?.',
  [2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1]],
 ['?##..?#?#????##..?#?#????##..?#?#????##..?#?#????##..?#?#??',
  [2, 4, 2, 4, 2, 4, 2, 4, 2, 4]],
 ['..#??##????###????#??..#??##????###????#??..#??##????###????#??..#??##????###????#??..#??##????###????#?',
  [6, 8, 6, 8, 6, 8, 6, 8, 6, 8]],
 ['???#..?#??????#..?#??????#..?#??????#..?#??????#..?#??',
  [

In [6]:
def generate_combinations(s):
    result = []

    def count_connected_hashes(s):
        counts = []
        current_count = 0

        for char in s:
            if char == '#':
                current_count += 1
            else:
                if current_count > 0:
                    counts.append(current_count)
                    current_count = 0

        if current_count > 0:
            counts.append(current_count)

        return counts

    def generate_helper(current_combination, remaining_chars):
        if not remaining_chars:
            result.append(count_connected_hashes(current_combination))
            return

        if remaining_chars[0] == '?':
            generate_helper(current_combination + '.', remaining_chars[1:])
            generate_helper(current_combination + '#', remaining_chars[1:])
        else:
            generate_helper(current_combination + remaining_chars[0], remaining_chars[1:])

    generate_helper('', s)
    return result

In [8]:
from tqdm import tqdm
ans = 0
for gears, config in tqdm(gear_rows):
    ans += sum([1 for x in generate_combinations(gears) if x == config])
ans

 17%|█████████████▊                                                                     | 1/6 [05:54<29:34, 354.90s/it]


KeyboardInterrupt: 

In [78]:
def generate_valid_combinations(row, row_hints):
    def dp(index, remaining_hints):
        if index == len(row):
            return 1 if remaining_hints == 0 else 0

        if (index, remaining_hints) in memo:
            return memo[(index, remaining_hints)]

        count = 0
        hint = row_hints[0] if remaining_hints > 0 else 0

        for i in range(index, min(index + hint, len(row))):
            if row[i] == '?' or row[i] == '#':
                count += dp(i + 1, remaining_hints - 1)

        memo[(index, remaining_hints)] = count
        return count

    memo = {}
    total_hints = sum(row_hints)
    total_cells = len(row)

    if total_cells != total_hints + len(row_hints) - 1:
        # Invalid combination, the sum of hints and separators doesn't match the total cells
        return []

    valid_combinations = []
    if dp(0, len(row_hints)) > 0:
        # Only proceed to generate combinations if a valid combination exists
        def backtrack(index, current_combination, remaining_hints):
            if index == total_cells:
                valid_combinations.append(current_combination)
                return

            hint = row_hints[0] if remaining_hints > 0 else 0

            for i in range(index, min(index + hint, total_cells)):
                if row[i] == '?' or row[i] == '#':
                    new_combination = (
                        current_combination[:i] +
                        '#' * (i + 1 - index) +
                        current_combination[i + 1:]
                    )
                    backtrack(i + 1, new_combination, remaining_hints - 1)

        backtrack(0, row, len(row_hints))

    return valid_combinations

# Example usage:
row = '???.###'
row_hints = [1, 1, 3]

valid_combinations = generate_valid_combinations(row, row_hints)
for combination in valid_combinations:
    print(combination)


In [12]:
valid_combinations

[]

In [79]:
def fill_single_combinations(row, row_hints):
    def backtrack(index, current_combination):
        if index == len(row_hints):
            filled_row.append(current_combination)
            return

        hint = row_hints[index]
        for i in range(len(current_combination) - hint + 1):
            segment = current_combination[i:i + hint]
            if all(cell == '?' or cell == '#' for cell in segment):
                new_combination = (
                    current_combination[:i] +
                    '#' * hint +
                    current_combination[i + hint:]
                )
                backtrack(index + 1, new_combination)

    filled_row = []
    backtrack(0, row)

    if len(filled_row) == 1:
        return filled_row[0]
    else:
        return row  # No unique filling found, return the original row

# Example usage:
row = '???.###'
row_hints = [1, 1, 3]

result = fill_single_combinations(row, row_hints)
print(result)


???.###


In [80]:
def generate_combinations_with_pruning(s, row_hints):
    result = []

    def count_connected_hashes(s):
        counts = []
        current_count = 0

        for char in s:
            if char == '#':
                current_count += 1
            else:
                if current_count > 0:
                    counts.append(current_count)
                    current_count = 0

        if current_count > 0:
            counts.append(current_count)

        return counts

    def validate_combination(combination):
        # Check if the combination satisfies the row hints
        counts = count_connected_hashes(combination)
        return counts == row_hints

    def generate_helper(current_combination, remaining_chars):
        if not remaining_chars:
            if validate_combination(current_combination):
                result.append(current_combination)
            return

        if remaining_chars[0] == '?':
            generate_helper(current_combination + '.', remaining_chars[1:])
            generate_helper(current_combination + '#', remaining_chars[1:])
        else:
            generate_helper(current_combination + remaining_chars[0], remaining_chars[1:])

    generate_helper('', s)
    return result

# Example usage:
row = '???.###????.###????.###????.###????.###'
row_hints = [1, 1, 3, 1, 1, 3]

combinations = generate_combinations_with_pruning(row, row_hints)
print(combinations)


[]


In [3]:
from tqdm import tqdm
ans = 0
for gears, config in tqdm(gear_rows):
    ans += len(generate_combinations_with_pruning(gears, config))
    print(ans)
ans

  0%|                                                                                       | 0/1000 [1:49:14<?, ?it/s]


KeyboardInterrupt: 

In [81]:
len(generate_combinations_with_pruning("?###??????????###????????", [3,2,1,3,2,1]))

150

In [7]:
32 / 4

8.0

In [8]:
32 * 8**3

16384

In [11]:
150 * (150 / 10)**3

506250.0

In [122]:
import helpers
import numpy as np

#helpers.npNicePrint(arr)

data = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""
#with open("input_12.txt", 'r') as f:
#    data = f.read().strip()

#grid = {(x,y): val for y, r in enumerate(data.split('\n')) for x, val in enumerate(r) if val == "#"}
gear_rows = []
for row in data.split("\n"):
    gears, config = row.split(" ")
    config = [int(x) for x in config.split(",")]
    gear_rows.append([gears, config])

#start_coord = np.array([k for k,v in grid.items() if v == "S"][0])
gear_rows

[['???.###', [1, 1, 3]],
 ['.??..??...?##.', [1, 1, 3]],
 ['?#?#?#?#?#?#?#?', [1, 3, 1, 6]],
 ['????.#...#...', [4, 1, 1]],
 ['????.######..#####.', [1, 6, 5]],
 ['?###????????', [3, 2, 1]]]

In [60]:
ans = 0
for i, (gears, config) in enumerate(gear_rows):
    print(i)
    pt1 = len(generate_combinations_with_pruning(gears, config))
    print(gears, config)
    gears = (gears + "?" + gears)
    config = config*2
    print(gears, config)
    pt2 = len(generate_combinations_with_pruning(gears, config))
    #print("done")
    #pt2 = len(generate_combinations_with_pruning("?" + gears + "?", config))
    #print(pt1, pt2, pt1 * pt2**4)
    ans += (pt2 * (pt2/pt1)**3)
    #ans += (pt1 * pt2**4)
ans

0
??????#??#?? [1, 1, 5, 1]
??????#??#?????????#??#?? [1, 1, 5, 1, 1, 1, 5, 1]
1
?#?#??##?#? [2, 5, 1]
?#?#??##?#???#?#??##?#? [2, 5, 1, 2, 5, 1]
2
????????.?#???#??##? [2, 1, 2, 1, 1, 6]
????????.?#???#??##??????????.?#???#??##? [2, 1, 2, 1, 1, 6, 2, 1, 2, 1, 1, 6]


KeyboardInterrupt: 

In [25]:
ans

3889.0

In [ ]:
generate_combinations_with_pruning(gears + "?" + gears, config*2)


In [ ]:
, [2, 1, 2, 1, 1, 6]]

In [35]:
len(generate_combinations_with_pruning('?###????????', [3,2,1]))

10

In [52]:
len(generate_combinations_with_pruning('?###???????????###?????????', [3,2,1, 3, 2,1]))

315

In [37]:
10 * 15**4

506250

In [71]:

generate_combinations_with_pruning("????????.?#???#??##?",[2, 1, 2, 1, 1, 6])

['..........##########', '.........###########']

In [72]:
 generate_combinations_backtracking("????????.?#???#??##?",[2, 1, 2, 1, 1, 6])

KeyboardInterrupt: 

In [116]:
word = "?###????????"
hint = [3,2,1]
len_normal = len(generate_combinations_with_pruning(word,hint))
generate_combinations_with_pruning(word,hint)


['.###....##.#',
 '.###...##..#',
 '.###...##.#.',
 '.###..##...#',
 '.###..##..#.',
 '.###..##.#..',
 '.###.##....#',
 '.###.##...#.',
 '.###.##..#..',
 '.###.##.#...']

In [123]:
ans = 0
for i, (gears, hint) in enumerate(gear_rows):
    print(i)
    len_normal = len(generate_combinations_with_pruning(gears,hint))
    len_end = len(generate_combinations_with_pruning(gears + "?",hint))
    ans += len_normal * len_end**4
ans

0
1
2
3
4
5


508301

In [117]:
len_end = len(generate_combinations_with_pruning(word + "?",hint))
generate_combinations_with_pruning(word + "?",hint)


['.###.....##.#',
 '.###....##..#',
 '.###....##.#.',
 '.###...##...#',
 '.###...##..#.',
 '.###...##.#..',
 '.###..##....#',
 '.###..##...#.',
 '.###..##..#..',
 '.###..##.#...',
 '.###.##.....#',
 '.###.##....#.',
 '.###.##...#..',
 '.###.##..#...',
 '.###.##.#....']

In [118]:

len_start = len(generate_combinations_with_pruning("?" + word,hint))
generate_combinations_with_pruning("?" + word,hint)

['..###....##.#',
 '..###...##..#',
 '..###...##.#.',
 '..###..##...#',
 '..###..##..#.',
 '..###..##.#..',
 '..###.##....#',
 '..###.##...#.',
 '..###.##..#..',
 '..###.##.#...']

In [119]:
4 * 8**4
len_normal * len_end**4

506250

In [89]:
generate_combinations_with_pruning("????????.?#???#??##??",[2,1,2,1,1,6])

['.##.#.##..#.#.######.',
 '##..#.##..#.#.######.',
 '##.#..##..#.#.######.',
 '##.#.##...#.#.######.']

In [88]:
generate_combinations_with_pruning("?????????.?#???#??##?",[2,1,2,1,1,6])

['..##.#.##..#.#.######',
 '.##..#.##..#.#.######',
 '.##.#..##..#.#.######',
 '.##.#.##...#.#.######',
 '##...#.##..#.#.######',
 '##..#..##..#.#.######',
 '##..#.##...#.#.######',
 '##.#...##..#.#.######',
 '##.#..##...#.#.######',
 '##.#.##....#.#.######',
 '##.#.##.#..#...######',
 '##.#.##.#..#..######.']

In [114]:
def is_valid_combination(row, hint):
    filled_count = 0
    current_group = 0

    for cell in row:
        if cell == '#':
            filled_count += 1
            if filled_count > hint[current_group]:
                return False
        elif cell == '.':
            if filled_count > 0:
                current_group += 1
                filled_count = 0

    if current_group != len(hint) - 1:
        return False

    return filled_count == hint[current_group]

def generate_combinations(row, hint, index=0, current=[]):
    if index == len(hint):
        if is_valid_combination(current, hint):
            yield ''.join(current)
        return

    for i in range(len(row) - hint[index] + 1):
        new_combination = current + ['.' * i + '#' * hint[index] + '.' * (len(row) - i - hint[index])]
        yield from generate_combinations(row, hint, index + 1, new_combination)

def count_valid_combinations(row, hint):
    return sum(1 for _ in generate_combinations(row, hint))

# Example usage:
row = "?????????.?#???#??##?"
hint = [2, 1, 2, 1, 1, 6]
result = count_valid_combinations(row, hint)
print(result)


KeyboardInterrupt: 

In [127]:
def generate_combinations(hint, remaining_space, current_combination, valid_combinations):
    if not hint and not remaining_space:
        valid_combinations.append(current_combination)
        return

    if not hint or not remaining_space:
        return

    block_size = hint[0]
    for i in range(1, len(remaining_space) + 1):
        if remaining_space[i - 1] == '.':
            new_combination = current_combination + ['.'] * (i - 1) + ['#'] * block_size
            new_remaining_space = remaining_space[i:]
            generate_combinations(hint[1:], new_remaining_space, new_combination, valid_combinations)

    generate_combinations(hint[1:], remaining_space, current_combination + ['.'], valid_combinations)

def count_valid_rows(row, hint):
    valid_combinations = []
    generate_combinations(hint, row, [], valid_combinations)

    count = 0
    for combination in valid_combinations:
        if ''.join(combination) == row:
            count += 1

    return count

# Example usage:
row = "?????????.?#???#??##?"
hint = [2, 1, 2, 1, 1, 6]

result = count_valid_rows(row, hint)
print(f"Total number of valid rows: {result}")


Total number of valid rows: 0


In [128]:
[x*"#" for x in hint]

['##', '#', '##', '#', '#', '######']

In [129]:
len(row)

21

In [133]:
def generate_block_combinations(blocks, row_length, fixed_pattern, current_string="", block_index=0):
    if block_index == len(blocks):
        # Check if the generated string satisfies the fixed pattern
        for char_mod, char_fixed in zip(current_string, fixed_pattern):
            if char_fixed != "?" and char_fixed != char_mod:
                return []  # This combination does not satisfy the fixed pattern

        return [current_string]

    valid_combinations = []

    # Try placing the block at the next valid position
    position = current_string.find(".", block_index)
    while position != -1:
        new_string = (
            current_string[:position] + blocks[block_index] + current_string[position + len(blocks[block_index]):]
        ).ljust(row_length, ".")

        # Recursive call for the next block
        combinations = generate_block_combinations(
            blocks, row_length, fixed_pattern, new_string, block_index + 1
        )

        valid_combinations.extend(combinations)

        # Find the next valid position
        position = current_string.find(".", position + 1)

    return valid_combinations


# Example usage
blocks = ['##', '#', '##', '#', '#', '######']
row_length = 21
fixed_pattern = "?????????.?#???#??##?"

combinations = generate_block_combinations(blocks, row_length, fixed_pattern)

for i, combination in enumerate(combinations, start=1):
    print(f"Combination {i}: {combination}")


In [140]:
def generate_strings(blocks, length, current_index=0, current_string='', separator='.'):
    # Base case: if the length is reached, return the current string
    if current_index == len(blocks):
        return [current_string[:-1]]  # Remove the trailing separator

    result = []

    # Try to add the current block to the current string
    block_length = len(blocks[current_index])
    if len(current_string) + block_length <= length:
        # Recursively call the function with the updated parameters
        next_strings = generate_strings(blocks, length, current_index + 1, current_string + blocks[current_index] + separator)

        # Append the results to the current result list
        result.extend(next_strings)

    # If possible, try to add the next block without a separator
    if len(current_string) + block_length <= length:
        next_strings = generate_strings(blocks, length, current_index + 1, current_string + blocks[current_index])
        result.extend(next_strings)

    return result

# Example usage
blocks = ['##', '#', '##', '#', '#', '######']
length = 21
result = generate_strings(blocks, length, separator='.')
print(result)


['##.#.##.#.#.######', '##.#.##.#.#.#####', '##.#.##.#.#######', '##.#.##.#.######', '##.#.##.##.######', '##.#.##.##.#####', '##.#.##.########', '##.#.##.#######', '##.#.###.#.######', '##.#.###.#.#####', '##.#.###.#######', '##.#.###.######', '##.#.####.######', '##.#.####.#####', '##.#.##########', '##.#.#########', '##.###.#.#.######', '##.###.#.#.#####', '##.###.#.#######', '##.###.#.######', '##.###.##.######', '##.###.##.#####', '##.###.########', '##.###.#######', '##.####.#.######', '##.####.#.#####', '##.####.#######', '##.####.######', '##.#####.######', '##.#####.#####', '##.###########', '##.##########', '###.##.#.#.######', '###.##.#.#.#####', '###.##.#.#######', '###.##.#.######', '###.##.##.######', '###.##.##.#####', '###.##.########', '###.##.#######', '###.###.#.######', '###.###.#.#####', '###.###.#######', '###.###.######', '###.####.######', '###.####.#####', '###.##########', '###.#########', '#####.#.#.######', '#####.#.#.#####', '#####.#.#######', '#####.#.####

In [225]:
data = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""
#with open("input_12.txt", 'r') as f:
#    data = f.read().strip()

#grid = {(x,y): val for y, r in enumerate(data.split('\n')) for x, val in enumerate(r) if val == "#"}
gear_rows = []
gear_rows_one = []
for row in data.split("\n"):
    gears, config = row.split(" ")
    config = [int(x) for x in config.split(",")]
    #gear_rows.append([repeat_and_join(gears, 4, "?") + "?", config*4])
    gear_rows.append([gears, config])
    

#start_coord = np.array([k for k,v in grid.items() if v == "S"][0])
gear_rows

[['???.###', [1, 1, 3]],
 ['.??..??...?##.', [1, 1, 3]],
 ['?#?#?#?#?#?#?#?', [1, 3, 1, 6]],
 ['????.#...#...', [4, 1, 1]],
 ['????.######..#####.', [1, 6, 5]],
 ['?###????????', [3, 2, 1]]]

In [238]:
import helpers
import numpy as np

#helpers.npNicePrint(arr)

data = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""
with open("input_12.txt", 'r') as f:
    data = f.read().strip()

#grid = {(x,y): val for y, r in enumerate(data.split('\n')) for x, val in enumerate(r) if val == "#"}
gear_rows = []
for row in data.split("\n"):
    gears, config = row.split(" ")
    config = [int(x) for x in config.split(",")]
    gear_rows.append([gears, config])

#start_coord = np.array([k for k,v in grid.items() if v == "S"][0])
gear_rows

[['??????#??#??', [1, 1, 5, 1]],
 ['?#?#??##?#?', [2, 5, 1]],
 ['????????.?#???#??##?', [2, 1, 2, 1, 1, 6]],
 ['#???#?????.?#?.', [2, 1, 2, 1]],
 ['?##..?#?#??', [2, 4]],
 ['..#??##????###????#?', [6, 8]],
 ['???#..?#??', [1, 1, 2]],
 ['.#????#?.???', [3, 2]],
 ['.?#??##?.?', [1, 3]],
 ['????????#?', [2, 1, 1]],
 ['.?##???#?##????.?', [7, 4, 1]],
 ['?##??.?#.?????', [5, 1, 4]],
 ['?????.#??#??', [1, 1, 1, 1]],
 ['?#.???.???.##?#?.', [1, 1, 1, 1, 5]],
 ['.#??.??.???.???.', [1, 1, 2, 1, 2]],
 ['.?.????.???', [1, 1, 3]],
 ['?????#??????????#', [6, 1, 6]],
 ['#?????#????', [1, 8]],
 ['.???.??#?.#????#?.??', [1, 1, 1, 7, 1]],
 ['.??#????##??#.#', [12, 1]],
 ['?#????##.#.??', [2, 4, 1, 2]],
 ['??.??#?#.#?', [1, 3, 2]],
 ['??#?.#????#?#?', [2, 9]],
 ['#??#????#?????', [4, 3, 1]],
 ['?##????.????', [3, 2]],
 ['?##??#?.#??#??#??#.?', [2, 2, 10]],
 ['??#?#??????#.#?#.?', [11, 3, 1]],
 ['??##.???????#??????.', [2, 1, 2, 4, 2]],
 ['????.?#.???#.????.?', [1, 1, 1, 4, 1, 1]],
 ['???????????#???.#??'

In [ ]:
from collections import deque
#blocks = deque(['##', '#', '##', '#', '#', '######'])

def add_block(blocks, max_len, solution = ""):
    ans = 0
    if not valid_pattern(pattern, solution):
        return 0
    if (length_to_come(blocks) + len(solution)) > max_len:
        return 0
    if len(solution) == max_len:
        #print(solution)
        return 1
    if len(blocks) > 0:
        block_new = blocks.copy()
        solution_add = solution + block_new.popleft()
        if len(solution_add) < max_len:
            solution_add += "."
        ans += add_block(block_new, max_len, solution_add)
    ans += add_block(blocks.copy(), max_len, solution + ".")
    return ans
ans = 0
for gear, hint in tqdm(gear_rows):
    #blocks = deque(["#"*x for x in hint])
    #pattern = gear
    #repeat_and_join(pattern, 4, "?") + "?", config*4
    #single = add_block(blocks, len(pattern))
    pattern = repeat_and_join(gear, 5, "?")
    blocks = deque(["#"*x for x in hint]*5)
    fours = add_block(blocks, len(pattern))
    ans += fours
    #ans += add_block(blocks, len(pattern))
    
    #pt1 = add_block(blocks, len(pattern))
    #blocks = deque(["#"*x for x in hint]*3)                
    #pattern = "?" + pattern + "?" + pattern + "?"
    #pattern = "?" + pattern + "?" + pattern + "?" + pattern + "?"
    #print(blocks, pattern)
    #pt2 = add_block(blocks, len(pattern))
    #print("done")
    #print(pt1, pt2)
    #print(pt1 * (pt2**2))
    #print(pt1, pt2, pt1 * pt2**4)
    #ans += 2* pt1 * (pt2)
    #ans += (pt2 * (pt2/pt1)**3)
ans

  3%|██                                                                       | 28/1000 [1:21:00<131:00:47, 485.23s/it]

In [231]:
8 * 2048

16384

In [ ]:
pattern + "?" + pattern + "?" + pattern + "?" + pattern + "?" + pattern

In [182]:
pattern = ".??..??...?##."

test = ".#.."

def valid_pattern(pattern, test):
    pattern = pattern[:len(test)]
    for char1, char2 in zip(pattern, test):
        if not char1 == "?":
            if not char1 == char2:
                return False
    return True

    
valid_pattern(pattern,test)

True

In [147]:
from collections import deque
t = deque(['##', '#', '##', '#', '#', '######'])
t2 = t.copy()
t.popleft()
t2

deque(['##', '#', '##', '#', '#', '######'])

In [148]:
t

deque(['#', '##', '#', '#', '######'])

In [158]:
def length_to_come(q):
    if len(q) == 0:
        return 0
    return(sum([len(x)+1 for x in q]) -1)

In [162]:
length_to_come(deque(["###"]))

3